In [1]:
import ipsuite as ips

2023-04-06 14:00:02.486983: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-06 14:00:02.558571: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-06 14:00:03.142055: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /data/fzills/tools/cuda/cuda-11.8/lib64:
2023-04-06 14:00:03.142099: W tensorflow/compiler/xla/str

2023-04-06 14:00:04,231 (DEBUG): Welcome to IPS - the Interatomic Potential Suite!


In [2]:
mapping = ips.geometry.BarycenterMapping(data=None)

with ips.Project() as project:
    data = ips.AddData(file="etoh_meoh_dimer.traj")

    test_data = ips.configuration_selection.RandomSelection(data=data, n_configurations=1000, name="test_data")
    train_data = ips.configuration_selection.RandomSelection(
        data=data, n_configurations=500, name="train_data",
        exclude_configurations=test_data.selected_configurations
    )
    val_data = ips.configuration_selection.RandomSelection(
        data=data, n_configurations=100, name="val_data",
        exclude_configurations=[test_data.selected_configurations, train_data.selected_configurations]
    )
    
    model = ips.models.MACE(data=train_data, test_data=val_data, max_num_epochs=250)

    predictions = ips.analysis.PredictWithModel(model=model, data=test_data)
    analysis = ips.analysis.AnalysePrediction(data=predictions)
    inter_intra_forces = ips.analysis.InterIntraForces(data=predictions)

    # TODO  rattle analysis, nve stability?

    etoh = ips.configuration_generation.SmilesToAtoms(smiles="CCO", name="EtOH")
    meoh = ips.configuration_generation.SmilesToAtoms(smiles="CO", name="MeOH")
    packmol = ips.configuration_generation.Packmol(data=[etoh.atoms, meoh.atoms], count=[10, 10], box=12)
    geopt = ips.calculators.ASEGeoOpt(calculator=model.calc, data=packmol.atoms, optimizer="FIRE")
    md = ips.calculators.ASEMD(data=geopt.atoms, model=model, friction=0.01, sampling_rate=1, steps=5000, temperature=250, time_step=0.5)


    ips.analysis.BoxScaleAnalysis(data=md, model=model, mapping=mapping)

project.run(repro=False)